In [1]:
import time
import numpy as np
import py_at_broker as pab
from IPython.display import clear_output
import math as math
import pandas as pd
import pylab as pl
import pickle as p
from IPython import display
import matplotlib.pyplot as plt
#%matplotlib inline
b = pab.broker()

%load_ext autoreload
%autoreload 2
from toolbox import kinematic_mapping as km
from toolbox import data_processing as dp
from toolbox import movementtoolbox as mt 
from toolbox import obstacle_detector as od


In [2]:
b.register_signal('franka_target_pos', pab.MsgType.target_pos)

Address broker could not acceppt the offered signal franka_target_pos. Is there another signal with the same name? Trying to reconnect
Reconnection to signal <franka_target_pos> successful


True

In [3]:
b.request_signal("franka_state", pab.MsgType.franka_state, True)

True

In [4]:
b.request_signal("franka_lidar", pab.MsgType.franka_lidar)

True

In [ ]:
#b.request_signal("realsense_images", pab.MsgType.realsense_image)

current_counter = 50020
sleep_time = 0.5
go_time = 0.15

msg = b.recv_msg("franka_state", 0)
print(msg.get_fnumber())

z = 0.7

inside_range = True
i = 10
polarity = 1
scaling=1.2

change_height = 0 




while z > 0.2 : 
    
    polarity_old = polarity
    
    timeth=i/scaling
    x, y = mt.create_spiral(timeth, 0.005)
    pos = np.array([x, y, z])
    frame=current_counter+1
    target_msg, current_counter = mt.create_message(counter=frame, pos = pos, go_time=go_time*0.3*i)

    
    msg = b.recv_msg("franka_state", 0)
    
    clear_output(wait = True)
    print(pos)
    print(polarity)
    b.send_msg('franka_target_pos', target_msg)
    time.sleep(sleep_time)
    
    i += polarity
    
    if x > 0.9: 
        polarity = -1
    elif i == 10: 
        polarity = 1
        
    if polarity_old - polarity != 0: 
        change_height += 1
    
    if (change_height>0) and (change_height%2 == 0): 
        z -= 0.1
        print('changed height to {}'.format(z))


In [5]:
lidar_msg = b.recv_msg("franka_lidar", 1)
print(lidar_msg)

In [6]:
def create_globe(center_x, center_y, center_z, radius): 
    phi = np.random.vonmises(0, 2)
    theta = np.random.vonmises(0, 2)
    
    x = radius*np.sin(theta)*np.cos(phi) + center_x
    y = radius*np.sin(theta)*np.sin(phi) + center_y
    z = radius*np.cos(theta) + center_z
    
    return np.array([x,y,z])


predictor = lp.lidar_predictor(binning_params = {'max_value':1, 'bin_lenght':0.01 }, 
                   model_names=['models/model_lidar_new{}.h5'.format(i) for i in range(9)])

In [17]:
def init_lidar_predictor(timesteps = 5): 
    predictor = lp.lidar_predictor(binning_params = {'max_value':1, 'bin_lenght':0.01 }, 
                   model_names=['models/lidar_models_proMp_and_random_5000Epochs/model_lidar_5000_epochs_{}.h5'.format(i) for i in range(9)])
    
    measurements = []
    predictions = []
    
    print('initializing measurements: ', end ='')
    for t in range(timesteps): 
        lidar_msg = b.recv_msg("franka_lidar", 0)
        state_msg = b.recv_msg("franka_state", 0)  
        prediction, measurement = predictor.predict_single(joint_data=state_msg.get_j_pos(), true_lidar=lidar_msg.get_data())
        predictions.append(prediction)
        measurements.append(measurement)
        time.sleep(0.5)
        print('.', end = '')
    print('done')
    return predictor, predictions, measurements

        
        
    

In [18]:
def calc_error(predictor, predictions, measurements, new_lidar, new_joints, threshold): 
    prediction, measurement = predictor.predict_single(joint_data=state_msg.get_j_pos(), true_lidar=lidar_msg.get_data())
    
    predictions.append(prediction)
    predictions.pop(0)
    
    measurements.append(measurement)
    measurements.pop(0)
    
    error = compare2threshold(np.array(predictions), np.array(measurements), threshold)
    return error, predictions, measurements

In [19]:
def compare2threshold(prediction, measurement, threshold): 
    # dim prediction = timestamps x lidars
    
    #prediction = np.reshape(prediction, (prediction.shape[0], prediction.shape[2]))
    #measurement = np.reshape(measurement, (measurement.shape[0], measurement.shape[1]))
    error = prediction-measurement # positive error means something is closer than expected
    error = np.divide(error, prediction)
    med_error = np.median(error, axis = 0) # calculate median over timestamps will ignore negantive and small values 
    #print(med_error)
    if (med_error>threshold).any():
        print('Deviation of {} in liadar {}.'.format(np.round(np.max(med_error), 2), np.argmax(med_error)))
        #print(med_error)
        return True, np.argmax(med_error)
    else: 
        return False, None

In [20]:
def check_approaching(measurements, index): 
    diff = np.median(np.diff(np.array(measurements)[:, index]))
    if diff < 0: 
        print('approaching')

In [21]:
%load_ext autoreload
%autoreload 2
from toolbox import lidar_predictor as lp
predictor, predictions, measurements = init_lidar_predictor()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Done loading model 0
Done loading model 1
Done loading model 2
Done loading model 3
Done loading model 4
Done loading model 5
Done loading model 6
Done loading model 7
Done loading model 8
initializing measurements: .....done


In [22]:
loc = od.localizer()

In [23]:
current_counter = 190
filename = 'unnecessary_recording.p'

cols = ['timestamp_franka', 'joint_pos', 'joint_vel', 'endeff_pos', 
        'timestamp_lidar', 'lidar_data']
df = pd.DataFrame(columns = cols)

state_msg = b.recv_msg("franka_state", 0)  
ref = state_msg.get_fnumber()

poses = [[0.3, -0.3, 0.7], [0.3, 0.3, 0.7], [0.7, -0.3, 0.7], [0.7, 0.3, 0.7]]
i = 0

save_and_send = 0    
lidars = []
joints = []

while True: 
    
    
    time.sleep(0.2)
    state_msg = b.recv_msg("franka_state", 0)
    lidar_msg = b.recv_msg("franka_lidar", 0)
    
    lidars.append(lidar_msg.get_data())
    joints.append(state_msg.get_j_pos())
    #print('reading lidar')
    
    #if state_msg.get_fnumber() == ref: 
    #    break

    if save_and_send > 0.8:
        #print('sending position')
        state_msg = b.recv_msg("franka_state", 0)  # then we get a signal from the state as fast as possible (tested)
        #camera_msg = b.recv_msg("realsense_images", 0)

        #pred = predictor.predict_single(np.ones(7), lidar_msg.get_data(), plot =False)
        joint_pos = state_msg.get_j_pos()
        lidar_measurement = lidar_msg.get_data()



        df2 = pd.DataFrame([state_msg.get_timestamp(), joint_pos, state_msg.get_j_vel(), state_msg.get_c_pos(), 
                         lidar_msg.get_timestamp(), lidar_measurement],cols)

        df = df.append(df2.T, ignore_index = True)

        ref = state_msg.get_fnumber()

        #noise = np.random.normal(loc = 0, scale = 0.1, size =3)
        #pos = np.array(poses[i%4]) + noise
        pos = create_globe(50, 0, 30, 30)#, phi_old, theta_old) 
        pos = pos/100.

        frame=current_counter+1
        target_msg, current_counter = mt.create_message(counter=frame, pos = pos, go_time=3)
        b.send_msg('franka_target_pos', target_msg)
        i+=1
        clear_output(wait = True)
        print(pos)
        print(i)
        
        
        prediction, measurement = predictor.predict_timeseries(joint_data=np.array(joints), true_lidar=np.array(lidars))
        error, idx = compare2threshold(prediction, measurement, 0.4)
        if error: 
            obst = loc.localize_obstacle(joint_pos, lidar_measurement, idx)
            print(obst)
            time.sleep(5)
            #print(measurement.shape)
            #check_approaching(measurement, idx)
            
        save_and_send = 0
        lidars = []
        joints = []
        

        
    save_and_send += 0.1
    #error, predictions, measurements = calc_error(predictor, predictions, measurements, \
                                                  #lidar_msg.get_data(), state_msg.get_j_pos(), 0.2)
    

p.dump(df, open(filename, 'wb'))
print('simulation stopped')    
    

[ 0.56452154 -0.23238411  0.47842252]
18
Deviation of 0.87 in liadar 3.
obstacle found in front left top
[0.542 0.017 0.872]


KeyboardInterrupt: 

In [ ]:
signal =  b.recv_msg("franka_lidar", 0).get_data()
joints = b.recv_msg("franka_state", 0).get_j_pos()




In [ ]:
frame=current_counter+1
pos = np.array([0.7, 0.2,0.5])
target_msg, current_counter = mt.create_message(counter=frame, pos = pos, go_time=3)
b.send_msg('franka_target_pos', target_msg)

In [ ]:
i=0
lidars=[]
joints=[]
while i < 5:
    time.sleep(0.1)
    state_msg = b.recv_msg("franka_state", 0)
    lidar_msg = b.recv_msg("franka_lidar", 0)
    
    lidars.append(lidar_msg.get_data())
    joints.append(state_msg.get_j_pos())
    i+=1

prediction, measurement = predictor.predict_timeseries(joint_data=np.array(joints), true_lidar=np.array(lidars))

error, idx = compare2threshold(prediction, measurement, 0.2)
if error: 
    obst = loc.localize_obstacle(np.array(joints)[-1, :], measurement[-1, :], idx)
    print(obst)
else: 
    print('nothing in my way')

    
plt.figure(figsize = (9,9))
T = prediction.shape[0]
for i in range(9): 
    plt.subplot(3,3,i+1)
    plt.scatter(np.arange(0,T,1), prediction[:, i], label = 'pred', s = 100, c = 'b')
    plt.scatter(np.arange(0,T,1), measurement[:, i], label = 'measure', s = 100, c = 'r')
    plt.title('Lidar ' + str(i))
    plt.ylim(0, 1.2)
plt.legend()
plt.show()

data = np.array([create_globe(70, 0, 30, 30) for i in range(1000)])
import matplotlib.pyplot as plt
%matplotlib notebook
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(xs = data[:, 0], ys=data[:, 1], zs=data[:, 2], zdir='z')
plt.savefig('points_on_a_globe.jpg')

    prediction, measurement = predictor.predict_single(joint_data=state_msg.get_j_pos(), true_lidar=lidar_msg.get_data())
    error = prediction[0, :]-measurement[:, 0]
    if np.max(error) > 0.3: 
        print('max err: ', np.max(error))
        print('err in lidar: ', np.argmax(error))
    

In [ ]:
lidar_msg = b.recv_msg("franka_lidar", 0)
data = lidar_msg.get_data()

In [ ]:
print(data)
print(lp.lidar_preprocessing(np.reshape(data, (len(data), 1)))[:, 0])

In [ ]:
measurement.shape